# Influenza A - all subtypes


### NCBI entrez

I initially attempted to label all segments using the annotations in the genbank file that I could obtain using Biopython's entrez tool - but this is very slow. It takes over 2min to download 100k records, and Influenza A has over 1M records. Additionally, the annotations are often misspelled and are not standardized - so I ended up with long lists of protein synonyms that were still not complete (see the attached config). Ultimately this lead to only ~110k sequences being annotated with a segment. 

### Blast

Attempt to use Blast to label sequences segments and subtypes using local alignment: 

```
makeblastdb -in influenza_segments.fasta -dbtype nucl -out influenza_db
blastn -query ../results/ncbi_dataset/data/genomic.fna  -db influenza_db -out results.txt -outfmt 6 #only results
blastn -query ../results/ncbi_dataset/data/genomic.fna  -db influenza_db -out results.txt -outfmt 7 #results and sequences with no hits
```

This leads to over 80k sequences that have no hits (see field at the bottom). I tried to tune the query to increase the number of hits, but this was not successful: 

```
blastn -query ../results/ncbi_dataset/data/genomic.fna  -db influenza_db -out results.txt -outfmt 6 -evalue 1e5 -max_target_seqs 1 -dust no -word_size 7 -max_hsps 1
```

In [14]:
from Bio import Entrez, SeqIO

# Set your email address (required by NCBI)
Entrez.email = "your_email@example.com"

for seg, accession_number in enumerate(["NC_026438.1", "NC_026435.1", "NC_026437.1", "NC_026433.1", "NC_026436.1", "NC_026434.1", "NC_026431.1", "NC_026432.1"]):

    # Fetch the sequence in FASTA format
    with Entrez.efetch(db="nucleotide", id=accession_number, rettype="fasta", retmode="text") as handle:
        record = SeqIO.read(handle, "fasta")
        print(f">H1N1_segment_{seg+1}")
        print(record.seq)

for seg, accession_number in enumerate(["NC_026422.1", "NC_026423.1", "NC_026424.1", "NC_026425.1", "NC_026426.1", "NC_026429.1", "NC_026427.1", "NC_026428.1"]):

    # Fetch the sequence in FASTA format
    with Entrez.efetch(db="nucleotide", id=accession_number, rettype="fasta", retmode="text") as handle:
        record = SeqIO.read(handle, "fasta")
        print(f">H7N9_segment_{seg+1}")
        print(record.seq)

for seg, accession_number in enumerate(["NC_007378.1", "NC_007375.1", "NC_007376.1", "NC_007374.1", "NC_007381.1", "NC_007382.1", "NC_007377.1", "NC_007380.1"]):

    # Fetch the sequence in FASTA format
    with Entrez.efetch(db="nucleotide", id=accession_number, rettype="fasta", retmode="text") as handle:
        record = SeqIO.read(handle, "fasta")
        print(f">H2N2_segment_{seg+1}")
        print(record.seq)

for seg, accession_number in enumerate(["NC_007373.1", "NC_007372.1", "NC_007371.1", "NC_007366.1", "NC_007369.1", "NC_007368.1", "NC_007367.1", "NC_007370.1"]):

    # Fetch the sequence in FASTA format
    with Entrez.efetch(db="nucleotide", id=accession_number, rettype="fasta", retmode="text") as handle:
        record = SeqIO.read(handle, "fasta")
        print(f">H3N2_segment_{seg+1}")
        print(record.seq)

for seg, accession_number in enumerate(["NC_004910.1", "NC_004911.1", "NC_004912.1", "NC_004908.1", "NC_004905.2", "NC_004909.1", "NC_004907.1", "NC_004906.1"]):

    # Fetch the sequence in FASTA format
    with Entrez.efetch(db="nucleotide", id=accession_number, rettype="fasta", retmode="text") as handle:
        record = SeqIO.read(handle, "fasta")
        print(f">H9N2_segment_{seg+1}")
        print(record.seq)


>H1N1_segment1
ATGGAGAGAATAAAAGAACTGAGAGATCTAATGTCGCAGTCCCGCACTCGCGAGATACTCACTAAGACCACTGTGGACCATATGGCCATAATCAAAAAGTACACATCAGGAAGGCAAGAGAAGAACCCCGCACTCAGAATGAAGTGGATGATGGCAATGAGATACCCAATTACAGCAGACAAGAGAATAATGGACATGATTCCAGAGAGGAATGAACAAGGACAAACCCTCTGGAGCAAAACAAACGATGCTGGATCAGACCGAGTGATGGTATCACCTCTGGCCGTAACATGGTGGAATAGGAATGGCCCAACAACAAGTACAGTTCATTACCCTAAGGTATATAAAACTTATTTCGAAAAGGTCGAAAGGTTGAAACATGGTACCTTCGGCCCTGTCCACTTCAGAAATCAAGTTAAAATAAGGAGGAGAGTTGATACAAACCCTGGCCATGCAGATCTCAGTGCCAAGGAGGCACAGGATGTGATTATGGAAGTTGTTTTCCCAAATGAAGTGGGGGCAAGAATACTGACATCAGAGTCACAGCTGGCAATAACAAAAGAGAAGAAAGAAGAGCTCCAGGATTGTAAAATTGCTCCCTTGATGGTGGCGTACATGCTAGAAAGAGAATTGGTCCGTAAAACAAGGTTTCTCCCAGTAGCCGGCGGAACAGGCAGTGTTTATATTGAAGTGTTGCACTTAACCCAAGGGACGTGCTGGGAGCAGATGTACACTCCAGGAGGAGAAGTGAGAAATGATGATGTTGACCAAAGTTTGATTATCGCTGCTAGAAACATAGTAAGAAGAGCAGCAGTGTCAGCAGACCCATTAGCATCTCTCTTGGAAATGTGCCACAGCACACAGATTGGAGGAGTAAGGATGGTGGACATCCTTAGACAGAATCCAACTGAGGAACAAGCCGTAGACATATGCAAGGCAGCAATAGGGTTGAGGATTAGCTCATCTTTCAGTTTTGGTGGGTTCA

In [27]:
from Bio import SeqIO

# Extract IDs from input FASTA
fasta_ids = {}

for record in SeqIO.parse("results/ncbi_dataset/data/genomic.fna", "fasta"):
    fasta_ids[record.id] = record.description

# Extract IDs from BLAST output
blast_ids = []
with open("segments/results.txt", "r") as blast_file:
    for line in blast_file:
        blast_ids.append(line.split("\t")[0])

print(blast_ids[0])
print(fasta_ids[blast_ids[0]])

print(len(fasta_ids.keys()))

# Find IDs with no hits
no_hits_ids = set(fasta_ids.keys()) - set(blast_ids)
print(len(no_hits_ids))

# Save IDs without hits
with open("no_hits_ids.txt", "w") as output_file:
    for no_hit_id in no_hits_ids:
        output_file.write(fasta_ids[no_hit_id] + "\n")

MH828535.1
MH828535.1 Influenza A virus (A/Viet nam/2548/2014(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
1136048
81284


## Nextclade sort

Cornelius then showed me nextclade sort - which is similar to Blast but faster and more customizable. It requires a minimizer_index file - which can be generated from the influenza reference sequences file I created above using the same code used by nextclade datasets (https://github.com/nextstrain/nextclade_data/blob/master/scripts/lib/minimizer.py).

Then the parameters can be tuned: 

```
nextclade sort -m influenza_segments.minimizer.json -r out.tsv  sample_influenza.fasta --max-score-gap 0.5 --min-score 0.01 --min-hits 1  --all-matches
```

In [ ]:
import copy

import numpy as np
from Bio.SeqRecord import SeqRecord, SeqIO
import json

# Increment this version, if there are changes in the algorithm.
# Backwards compatibility must be ensured to not break client code: all versions must be computed and reffered to
# from the dataset's server 'index.json' file.
MINIMIZER_ALGO_VERSION = "1"

# Increment this version, if there are changes in the layout of the output file.
MINIMIZER_JSON_SCHEMA_VERSION = "3.0.0"

# What is this?
MAGIC_NUMBER_K = 17

# minimizer cutoff. The max is 1<<32 - 1, so with 28 uses roughly 1/16 of all kmers
CUTOFF = 1 << 32 - 1


# from lh3
def invertible_hash(x):
    m = (1 << 32) - 1
    x = (~x + (x << 21)) & m
    x = x ^ (x >> 24)
    x = (x + (x << 3) + (x << 8)) & m
    x = x ^ (x >> 14)
    x = (x + (x << 2) + (x << 4)) & m
    x = x ^ (x >> 28)
    x = (x + (x << 31)) & m
    return x


# turn a kmer into an integer
def get_hash(kmer):
    x = 0
    j = 0
    for i, nuc in enumerate(kmer):
        if i % 3 == 2:
            continue  # skip every third nucleotide to pick up conserved patterns
        if nuc not in "ACGT":
            return CUTOFF + 1  # break out of loop, return hash above cutoff
        else:  # A=11=3, C=10=2, G=00=0, T=01=1
            if nuc in "AC":
                x += 1 << j
            if nuc in "AT":
                x += 1 << (j + 1)
        j += 2

    return invertible_hash(x)


def get_ref_search_minimizers(seq: SeqRecord, k=MAGIC_NUMBER_K):
    seq_str = preprocess_seq(seq)
    minimizers = []
    # we know the rough number of minimizers, so we can pre-allocate the array if needed
    for i in range(len(seq_str) - k):
        kmer = seq_str[i : i + k]
        mhash = get_hash(kmer)
        if (
            mhash < CUTOFF
        ):  # accept only hashes below cutoff --> reduces the size of the index and the number of look-ups
            minimizers.append(mhash)
    return np.unique(minimizers)


def make_ref_search_index(refs):
    # collect minimizers for each reference sequence first
    minimizers_by_reference = list()
    for name, ref in sorted(refs.items()):
        minimizers = get_ref_search_minimizers(ref)
        minimizers_by_reference.append(
            {
                "minimizers": minimizers,
                "meta": {"name": name, "length": len(ref.seq), "nMinimizers": len(minimizers)},
            }
        )

    # construct an index where each minimizer maps to the references it contains via a bit set (here boolean np array)
    index = {"minimizers": {}, "references": []}
    for ri, minimizer_set in enumerate(minimizers_by_reference):
        for m in minimizer_set["minimizers"]:
            if m not in index["minimizers"]:
                index["minimizers"][m] = []
            index["minimizers"][m].append(ri)

        # reference will be a list in same order as the bit set
        index["references"].append(minimizer_set["meta"])

    normalization = np.array([x["length"] / x["nMinimizers"] for x in index["references"]])

    return {
        "schemaVersion": MINIMIZER_JSON_SCHEMA_VERSION,
        "version": MINIMIZER_ALGO_VERSION,
        "params": {
            "k": MAGIC_NUMBER_K,
            "cutoff": CUTOFF,
        },
        **index,
        "normalization": normalization,
    }


def preprocess_seq(seq: SeqRecord) -> str:
    return str(seq.seq).upper().replace("-", "")


def serialize_ref_search_index(index):
    index = copy.deepcopy(index)
    index["minimizers"] = {str(k): v for k, v in index["minimizers"].items()}
    index["normalization"] = index["normalization"].tolist()
    return index


def to_bitstring(arr) -> str:
    return "".join([str(x) for x in arr])


# Load the reference sequences
refs = {rec.id: rec for rec in SeqIO.parse("influenza_segments.fasta", "fasta")}

index = make_ref_search_index(refs)

serialized = serialize_ref_search_index(index)

with open("influenza_segments.minimizer.json", "w") as f:
    json.dump(serialized, f)

### Future Work / Ingest Updates

1. Change from nextclade align to sort with a minimizer index - this should be added to the seqs directory (also created for CCHF)
2. Improve the Influenza reference sequences list used in by nextclade sort - ideally this would include reference sequences for all known HA, NA subtypes -> there are approx. 13x12, with each 8 segments, i.e. a total of 1048 reference segments. I don't want to have to have to manually annotate this and sadly the NCBI virus webpage isn't easily scrapped (I tried using beautiful soup), so I will most likely download all assemblies as these include annotated segments and choose the first fully annotated genome of each subtype as the reference - first starting with the refSeq database (sadly they only have 7 full reference genomes, and this includes two different H1N1 and H5N1 reference assemblies). Then I will search for the rest in genbank.
```
datasets download genome taxon 11320 --assembly-source refseq
datasets download genome taxon 11320 --assembly-source genbank
```
3. Change the ingest filter that filters sequences (e.g. H5N1 sequences) based on a field in the fasta header to filter sequences based on the results of nextclade sort - this should be more sensitive. 
4. Add a "known" grouping field - e.g. a list of sequences where the groups are known. In influenza we have ~80k assemblies - this is a source of truth for grouping segments - we know for sure that all sequences in an assembly should be grouped. Before grouping based on a heuristic we should group based on the results of the assembly download - sadly the accessions of nucleotide sequences in an assembly cannot be seen from the summary alone: e.g. `datasets summary genome accession GCF_001343785.1`, so we will need to download all assemblies and parse the contents to obtain these groups - this can only be seen inside the downloaded fasta (also note the downloaded assemblies do not include metadata information so we still need to download all nucleotide sequences with metadata). Note this will have no affect on CCHF as there is only one assembly there - the reference genome which we already group together. 
